In [1]:
!nvidia-smi

Wed Feb 21 06:53:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:0D:00.0 Off |                  Off |
| 30%   28C    P8              20W / 300W |   7622MiB / 49140MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import OneCycleLR
import os
from PIL import Image
import numpy as np
import random
from tqdm import tqdm

from base64 import b64encode
import numpy as np
import json
import cv2
import subprocess

# For video display:
from IPython.display import HTML
from pathlib import Path
from torch import autocast
from torchvision import transforms as tfms
import pytorch_lightning as pl
from transformers import logging #CLIPTextModel, CLIPTokenizer,
from transformers import BlipProcessor, BlipForConditionalGeneration

# torch.manual_seed(1)

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline

/opt/conda/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
os.getcwd()

'/'

In [4]:
# !git clone https://github.com/lllyasviel/ControlNet.git
!git clone https://github.com/anilbhatt1/ControlNet.git

Cloning into 'ControlNet'...
remote: Enumerating objects: 1398, done.
remote: Counting objects: 100% (477/477), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 1398 (delta 420), reused 408 (delta 393), pack-reused 921
Receiving objects: 100% (1398/1398), 122.40 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (637/637), done.


In [6]:
# Do huggingface-cli login and add token via vastai terminal before doing this
downloaded_model_path = hf_hub_download(repo_id="stabilityai/stable-diffusion-2-1-base",
                                        filename="v2-1_512-ema-pruned.ckpt",
                                        use_auth_token=True)
print(downloaded_model_path)

v2-1_512-ema-pruned.ckpt:   0%|          | 0.00/5.21G [00:00<?, ?B/s]

/root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1-base/snapshots/5ede9e4bf3e3fd1cb0ef2f7a3fff13ee514fdf06/v2-1_512-ema-pruned.ckpt


In [6]:
os.getcwd()

'/'

In [ ]:
#Copy file using gdown and apt install unzip before running this
!unzip -q '/fill50k.zip'

In [7]:
!ls '/fill50k/source' | wc -l
!ls '/fill50k/target' | wc -l

50000
50000


In [4]:
class MyDataset(Dataset):
    def __init__(self):
        self.data = []
        with open('/prompt_10k.json', 'rt') as f:
            for line in f:
                self.data.append(json.loads(line))
        self.width = 512
        self.height = 512

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        source_filename = item['source']
        target_filename = item['target']
        prompt = item['prompt']

        source = cv2.imread('/fill50k/' + source_filename)
        target = cv2.imread('/fill50k/' + target_filename)

        source = cv2.resize(source, (self.width, self.height), interpolation=cv2.INTER_NEAREST)
        target = cv2.resize(target, (self.width, self.height), interpolation=cv2.INTER_NEAREST)

        # Do not forget that OpenCV read images in BGR order.
        source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
        target = cv2.cvtColor(target, cv2.COLOR_BGR2RGB)

        # Normalize source images to [0, 1].
        source = source.astype(np.float32) / 255.0

        # Normalize target images to [-1, 1].
        target = (target.astype(np.float32) / 127.5) - 1.0

        return dict(jpg=target, txt=prompt, hint=source)

In [5]:
dataset = MyDataset()
print(len(dataset))

item = dataset[5234]
jpg = item['jpg']
txt = item['txt']
hint = item['hint']
print(txt)
print(jpg.shape)
print(hint.shape)

10000
fuchsia circle with yellow background
(512, 512, 3)
(512, 512, 3)


In [10]:
!mkdir '/models'

In [7]:
!cp '/root/.cache/huggingface/hub/models--stabilityai--stable-diffusion-2-1-base/snapshots/5ede9e4bf3e3fd1cb0ef2f7a3fff13ee514fdf06/v2-1_512-ema-pruned.ckpt' '/models'

In [8]:
%cd '/ControlNet'
! ls

/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/ControlNet
LICENSE			       gradio_pose2image.py
README.md		       gradio_scribble2image.py
__pycache__		       gradio_scribble2image_interactive.py
annotator		       gradio_seg2image.py
cldm			       image_log
config.py		       ldm
docs			       lightning_logs
environment.yaml	       models
font			       share.py
github_page		       test_imgs
gradio_annotator.py	       tool_add_control.py
gradio_canny2image.py	       tool_add_control_sd21.py
gradio_depth2image.py	       tool_transfer_control.py
gradio_fake_scribble2image.py  tutorial_dataset.py
gradio_hed2image.py	       tutorial_dataset_test.py
gradio_hough2image.py	       tutorial_train.py
gradio_normal2image.py	       tutorial_train_sd21.py


In [9]:
!python3 tool_add_control_sd21.py '/models/v2-1_512-ema-pruned.ckpt' '/models/control_sd21_512_ini.ckpt'

logging improved.
create_model-config : {'model': {'target': 'cldm.cldm.ControlLDM', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'use_checkpoint': True, 'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head_channels': 64, 'use_spatial_transformer': True, 'use_linear_in_transformer': True, 'transformer_depth': 1, 'context_dim': 1024, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'use_checkpoint': True, 'image_s

In [10]:
os.chdir('/ControlNet')
os.getcwd()

'/ControlNet'

In [11]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from tutorial_dataset import MyDataset
from cldm.logger import ImageLogger

In [12]:
from cldm.model import create_model, load_state_dict

In [13]:
resume_path = '/models/control_sd21_512_ini.ckpt'
batch_size = 8
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False
# First use cpu to load models. Pytorch Lightning will automatically move it to GPUs.
model = create_model('./models/cldm_v21.yaml').cpu()

create_model-config : {'model': {'target': 'cldm.cldm.ControlLDM', 'params': {'linear_start': 0.00085, 'linear_end': 0.012, 'num_timesteps_cond': 1, 'log_every_t': 200, 'timesteps': 1000, 'first_stage_key': 'jpg', 'cond_stage_key': 'txt', 'control_key': 'hint', 'image_size': 64, 'channels': 4, 'cond_stage_trainable': False, 'conditioning_key': 'crossattn', 'monitor': 'val/loss_simple_ema', 'scale_factor': 0.18215, 'use_ema': False, 'only_mid_control': False, 'control_stage_config': {'target': 'cldm.cldm.ControlNet', 'params': {'use_checkpoint': True, 'image_size': 32, 'in_channels': 4, 'hint_channels': 3, 'model_channels': 320, 'attention_resolutions': [4, 2, 1], 'num_res_blocks': 2, 'channel_mult': [1, 2, 4, 4], 'num_head_channels': 64, 'use_spatial_transformer': True, 'use_linear_in_transformer': True, 'transformer_depth': 1, 'context_dim': 1024, 'legacy': False}}, 'unet_config': {'target': 'cldm.cldm.ControlledUnetModel', 'params': {'use_checkpoint': True, 'image_size': 32, 'in_chan

In [14]:
model.load_state_dict(load_state_dict(resume_path, location='cpu'))
model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control

Loaded state_dict from [/models/control_sd21_512_ini.ckpt]


In [15]:
batch_size = 16
dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = ImageLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(accelerator="auto", precision=32, callbacks=[logger], max_epochs=5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [16]:
trainer.fit(model, dataloader)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /ControlNet/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                   | Params
-------------------------------------------------------------
0 | model             | DiffusionWrapper       | 865 M 
1 | first_stage_model | AutoencoderKL          | 83.7 M
2 | cond_stage_model  | FrozenOpenCLIPEmbedder | 354 M 
3 | control_model     | ControlNet             | 364 M 
------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


Data shape for DDIM sampling is (4, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps



DDIM Sampler: 100%|██████████| 50/50 [00:24<00:00,  2.01it/s]


OSError: [Errno 28] No space left on device